In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import when
from pyspark.sql.types import StructType, StructField, IntegerType
import datetime
import os

master = "spark://zy-ubuntu:7077"  
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--master {master} --driver-memory 4g --total-executor-cores 8 --executor-memory 8g --packages org.postgresql:postgresql:42.1.1 pyspark-shell'

In [2]:
spark = SparkSession.builder \
    .appName("casts") \
    .getOrCreate()

# set config to read from minio
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key", "admin")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "password")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://192.168.0.188:9000")  # must use IP address
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
spark.sparkContext._jsc.hadoopConfiguration().set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.sparkContext._jsc.hadoopConfiguration().set("spark.hadoop.fs.s3a.path.style.access", "true")
spark.sparkContext._jsc.hadoopConfiguration().set("-Dcom.amazonaws.services.s3.enableV4", "true")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.multipart.size", "104857600")        

In [3]:
title_principals_df = spark.read.csv("s3a://imdb/2021-06/06/title.principals.tsv", sep=r'\t', header=True)  
title_principals_df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- ordering: string (nullable = true)
 |-- nconst: string (nullable = true)
 |-- category: string (nullable = true)
 |-- job: string (nullable = true)
 |-- characters: string (nullable = true)



In [4]:
title_principals_df = title_principals_df.drop('ordering').drop('job').drop('category')

title_principals_df = title_principals_df.withColumn("characters", F.regexp_replace(F.col("characters"), '[\[\]\"]', "").alias("replaced"))
title_principals_df = title_principals_df.withColumn('characters', F.explode(F.split('characters', ',')))
title_principals_df = title_principals_df.withColumnRenamed('characters', 'character')
title_casts_df = title_principals_df.filter(
        (F.col('category') == 'self') | 
        (F.col('category') == 'actor') | 
        (F.col('category') == 'actress'))
# title_casts_df.show(truncate=False)   

In [5]:
# read tsv file into df
name_basics_df = spark.read.csv("s3a://imdb/2021-06/06/name.basics.tsv", sep=r'\t', header=True)  

# rename column
name_basics_df = name_basics_df.withColumnRenamed('primaryName', 'name')

# calculate age from birth year and death year
name_basics_df = name_basics_df.withColumn("age", when((F.col("birthYear") != '\\N') & (F.col("deathYear") != '\\N'), (F.col('deathYear').cast(IntegerType()) - F.col('birthYear').cast(IntegerType()))).when((F.col("birthYear") != '\\N') & (F.col("deathYear") == '\\N'), (datetime.datetime.now().year - F.col('birthYear')).cast(IntegerType())).otherwise(None))

# create is_alive column based on conditions
is_alive_col = F.when(
    (F.col("birthYear") != '\\N') & (F.col("deathYear") != '\\N'), False
).when((F.col("birthYear") != '\\N') & (F.col("deathYear") == '\\N'), True).otherwise(None)

name_basics_df = name_basics_df.withColumn('is_alive', is_alive_col)

# drop unused columns
name_basics_df_dropped = name_basics_df.drop('primaryProfession', 'knownForTitles', 'birthYear', 'deathYear')

In [6]:
df_casts = title_casts_df.join(name_basics_df_dropped, ['nconst'])
df_casts = df_casts.withColumn('character', F.when(F.col('character') == '\\N', F.lit(None)).otherwise(F.col('character')))

# df_casts.show()

In [7]:
df_upload = df_casts.drop('tconst').drop('character').drop_duplicates(['nconst'])
# df_upload.show()
df_upload.printSchema()

root
 |-- nconst: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- is_alive: boolean (nullable = true)



In [8]:
# insert df into dim_casts table
df_upload.write.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='casts',
      user='admin',
      password='password'
      ).mode('append').save()

In [9]:
titles_df = spark.read.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='titles',
      user='admin',
      password='password'
      ).load()
titles_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- download_date_id: integer (nullable = true)
 |-- tconst: string (nullable = true)
 |-- type: string (nullable = true)
 |-- primary_title: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- is_adult: boolean (nullable = true)
 |-- start_year: short (nullable = true)
 |-- end_year: short (nullable = true)
 |-- runtime_minutes: short (nullable = true)
 |-- av_rating: float (nullable = true)
 |-- num_votes: integer (nullable = true)
 |-- genre_1: string (nullable = true)
 |-- genre_2: string (nullable = true)
 |-- genre_3: string (nullable = true)



In [10]:
titles_id_df = titles_df.select('id', 'tconst')
titles_id_df = titles_id_df.withColumnRenamed('id', 'title_id')
# title_desc_id_df.show()

In [11]:
casts_from_pg_df = spark.read.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='casts',
      user='admin',
      password='password'
      ).load()
casts_from_pg_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: short (nullable = true)
 |-- is_alive: boolean (nullable = true)
 |-- nconst: string (nullable = true)



In [12]:
df_casts.printSchema()
titles_id_df.printSchema()

root
 |-- nconst: string (nullable = true)
 |-- tconst: string (nullable = true)
 |-- character: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- is_alive: boolean (nullable = true)

root
 |-- title_id: integer (nullable = true)
 |-- tconst: string (nullable = true)



In [13]:
casts_id_df = casts_from_pg_df.select('id', 'nconst')
casts_id_df = casts_id_df.withColumnRenamed('id', 'cast_id')

In [14]:
df_casts_composite_table = df_casts.select('nconst', 'tconst', 'character')

In [15]:
df_tmp = casts_id_df.join(df_casts_composite_table, ['nconst'])

In [19]:
df_titles_casts = df_tmp.join(titles_id_df, ['tconst'])
df_titles_casts = df_titles_casts.drop('nconst').drop('tconst')
df_titles_casts.printSchema()

root
 |-- cast_id: integer (nullable = true)
 |-- character: string (nullable = true)
 |-- title_id: integer (nullable = true)



In [20]:
# insert df into titles_casts table
df_titles_casts.write.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='titles_casts',
      user='admin',
      password='password'
      ).mode('append').save()